In [1]:
import cv2
import os
import numpy as np
import pandas as pd
import spacy
from spacy import displacy
import pytesseract
from pytesseract import Output
import matplotlib.pyplot as plt
pytesseract.pytesseract.tesseract_cmd="/home/aman/anaconda3/envs/tallyInvoiceParser.env/bin/tesseract"
os.environ['TESSDATA_PREFIX'] = "/home/aman/anaconda3/envs/tallyInvoiceParser.env/share/tessdata"
import import_ipynb
from textPreProcessor import *
from imagePreProcessor import *
from TesseractModify import *
from datetime import datetime
import math

2023-06-13 17:00:16.872138: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


importing Jupyter notebook from textPreProcessor.ipynb
Original Date ----> After Processing 

5-12-22
True
2-2-2023
True
5-12-2023
True
5-05-2023
True
5-05-2023
True
10-24-2022
True
10-24-2022
True
22-05-2023
True
21-02-23
True
5-12-22
True
31-08-2022
True
importing Jupyter notebook from imagePreProcessor.ipynb
importing Jupyter notebook from TesseractModify.ipynb


In [2]:
imagesFolderPath = '/home/aman/Documents/Tally/DocumentAI/Code/Tesseract-Modelv4/Dataset/SampleImages/'

In [3]:
def preProcessDataFrame(df, image):
    #drop null values
    df.dropna(inplace=True)
    
    
    #rows having text as a stopword removed
    indexesToDrop = []
    stopwords = ["", " ", "  ", "/", ":", "-", ".", ",", "\n", "\t", "\\", "(", ")", "[", "]", "{", "}", "*", "&", "%", "$", "#", "|"]
    for index in df.index:
        if df["text"][index] in stopwords:
            indexesToDrop.append(index)   
    df.drop(indexesToDrop, inplace=True)
    
    #resetting the index
    df.reset_index(inplace=True)
    df.drop(columns=["index"], inplace=True)
    
    #creating new columns for centroid information
    df["x"] = df["left"] + df["width"] 
    df["y"] = df["top"] + df["height"]
    
    df["PageHeight"] = image.shape[0]
    df["PageWidth"] = image.shape[1]
    
    df["isTop"] = 0
    df["isBottom"] = 0
    df["isRight"] = 0
    df["isLeft"] = 0

In [4]:
def findDate(dateDF):
    dates = []
    
    format = "%d-%m-%Y"
    for index in dateDF.index:
        text = dateDF["text"][index]
        res = True
        try:
            res = bool(datetime.strptime(text, format))
        except ValueError:
            res = False
        if str(res)=="True":
            dates.append(dateDF["text"][index])
            
    format = "%m-%d-%Y"
    for index in dateDF.index:
        text = dateDF["text"][index]
        res = True
        try:
            res = bool(datetime.strptime(text, format))
        except ValueError:
            res = False
        if str(res)=="True":
            dates.append(dateDF["text"][index])
            
    format = "%y-%m-%d"
    for index in dateDF.index:
        text = dateDF["text"][index]
        res = True
        try:
            res = bool(datetime.strptime(text, format))
        except ValueError:
            res = False
        if str(res)=="True":
            dates.append(dateDF["text"][index])
            
    return dates

In [5]:
def extractDateDataFrame(df):
    dateDF = df.copy()
    for index in dateDF.index:
        text = dateDF["text"][index]
        dateDF["text"][index] = preProcessText(text)
    dates = findDate(dateDF)
    return dateDF, dates
    
        

In [6]:
def returnDist(x1, y1, x2, y2):
    p = [x1, y1]
    q = [x2, y2]
    return math.dist(p, q)

In [7]:
def allocateNeighbours(df, neighbourFields):
    for index in df.index:
        x1 = df["x"][index]/df["PageWidth"][index]
        y1 = df["y"][index]/df["PageHeight"][index]
        block = df["block_num"][index]
        neighbours = {}
        for index2 in df.index:
            x2 = df["x"][index2]/df["PageWidth"][index2]
            y2 = df["y"][index2]/df["PageHeight"][index2]
            block2 = df["block_num"][index2]
            dist = returnDist(x1, y1, x2, y2)
            if dist<0.2:
                neighbours[df["text"][index2].lower()]=dist
            if abs(y1-y2)<0.2 and abs(block2-block)<=1<=1:
                neighbours[df["text"][index2].lower()] = dist
        for n in neighbourFields:
            if n in neighbours:
                df[n][index]=neighbours[n]

In [8]:
def addNeighbours(df):
    neighbourFields = ["date", "dated", "invoice", "delivery", "order", "due", "payment", "tax", "bill", "receipt", "issue"]
    for col in neighbourFields:
        df[col]=0
    allocateNeighbours(df, neighbourFields)
    

In [9]:
def findDateDF(dates, dateDF):
    indexes = []
    for date in dates:
        i = dateDF[dateDF["text"]==date].index
        for index in i:
            indexes.append(index)
    return indexes

In [10]:
def dropIndexes(indexes, df):
    indexesToDrop=[]
    for index in df.index:
        if index not in indexes:
            indexesToDrop.append(index)
    df.drop(indexesToDrop, inplace=True)

In [11]:
def create(imagePath):
    image = cv2.imread(imagePath)
    preProcessedImage = preProcessImage(imagePath)
    data = pytesseract.image_to_data(preProcessedImage, output_type=Output.DATAFRAME)
    processData(data)
    df = pd.DataFrame(data)
    preProcessDataFrame(df, preProcessedImage)
    dateDF, dates = extractDateDataFrame(df)
    addNeighbours(df)
    indexes = findDateDF(dates, dateDF)
    resultDF = df.copy()
    dropIndexes(indexes, resultDF)
    return resultDF
    
    

In [12]:
def batchCreate(folderPath):
    os.chdir(folderPath)
    images = os.listdir()
    dfList = []
    for imagePath in images:
        #print(imagePath)
        df = create(imagePath)
        df["imageName"] = imagePath
        df["output"] = 0
        dfList.append(df)
    return dfList

In [13]:
dfList = batchCreate(imagesFolderPath)

/tmp/ipykernel_21707/2943746453.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = preProcessText(text)
/tmp/ipykernel_21707/1269700397.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[n][index]=neighbours[n]
/tmp/ipykernel_21707/2943746453.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = preProcessText(text)
/tmp/ipykernel_21707/1269700397.py:18: SettingWithCopyWarning: 
A val

/tmp/ipykernel_21707/2943746453.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = preProcessText(text)
/tmp/ipykernel_21707/1269700397.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[n][index]=neighbours[n]
/tmp/ipykernel_21707/2943746453.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = preProcessText(text)
/tmp/ipykernel_21707/1269700397.py:18: SettingWithCopyWarning: 
A val

/tmp/ipykernel_21707/2943746453.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = preProcessText(text)
/tmp/ipykernel_21707/1269700397.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[n][index]=neighbours[n]
/tmp/ipykernel_21707/2943746453.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = preProcessText(text)
/tmp/ipykernel_21707/1269700397.py:18: SettingWithCopyWarning: 
A val

/tmp/ipykernel_21707/2943746453.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = preProcessText(text)
/tmp/ipykernel_21707/1269700397.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[n][index]=neighbours[n]
/tmp/ipykernel_21707/2943746453.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = preProcessText(text)
/tmp/ipykernel_21707/2943746453.py:5: SettingWithCopyWarning: 
A valu

In [15]:
len(dfList)

53

In [16]:
resultDF = pd.concat(dfList)

In [17]:
resultDF.shape

(50, 28)

In [18]:
len(resultDF["imageName"].unique())

30

In [19]:
resultDF

,block_num,left,top,width,height,conf,text,x,y,PageHeight,...,delivery,order,due,payment,tax,bill,receipt,issue,imageName,output
117,7,300,1105,155,23,96.191299,02.04.2023,455,1128,2339,...,0.000000,0.151753,0.000000,0.000000,0.528837,0.000000,0.000000,0.000000,20.jpeg,0
123,7,1379,1146,155,23,95.732468,02.04.2023,1534,1169,2339,...,0.000000,0.804302,0.000000,0.000000,0.131721,0.000000,0.000000,0.000000,20.jpeg,0
180,20,873,1833,106,18,69.672714,"02/04/2023,",979,1851,2339,...,0.000000,0.000000,0.000000,0.162403,0.421051,0.000000,0.000000,0.000000,20.jpeg,0
204,26,873,1904,106,18,96.788101,"02/04/2023,",979,1922,2339,...,0.000000,0.000000,0.000000,0.132623,0.000000,0.000000,0.000000,0.000000,20.jpeg,0
74,9,588,897,88,29,71.175232,"9/10/2022,",676,926,2339,...,0.000000,0.000000,0.000000,0.148762,0.000000,0.000000,0.000000,0.000000,23.jpeg,0
75,9,745,898,92,28,64.239807,"16-10-2022,",837,926,2339,...,0.000000,0.000000,0.000000,0.139577,0.000000,0.000000,0.000000,0.000000,23.jpeg,0
91,9,588,1037,84,19,94.234421,9/10/2022,672,1056,2339,...,0.000000,0.000000,0.000000,0.102512,0.000000,0.000000,0.000000,0.000000,23.jpeg,0
92,9,745,1031,88,32,84.635925,16-10-2022,833,1063,2339,...,0.000000,0.000000,0.000000,0.083202,0.000000,0.000000,0.000000,0.000000,23.jpeg,0
13,4,84,320,113,19,70.813957,04/14/2022,197,339,2200,...,0.000000,0.000000,0.000000,0.000000,0.050256,0.094122,0.000000,0.000000,18.jpeg,0
19,9,1216,328,127,22,73.269348,15/04/2022,1343,350,2200,...,0.000000,0.000000,0.000000,0.410680,0.000000,0.000000,0.252860,0.000000,39.jpeg,0


In [ ]:
resultDF.reset_index(inplace=True)

In [ ]:
resultDF.drop(columns=["index"], inplace=True)

In [ ]:
resultDF.head()

In [ ]:
pwd

In [ ]:
os.chdir('/home/aman/Documents/Tally/DocumentAI/Code/Tesseract-Modelv4/Dataset/')

In [ ]:
pwd

In [ ]:
resultDF.to_csv("tesseract.csv")